<a href="https://colab.research.google.com/github/Menager49/PY-net-elec/blob/main/Projet_Net_Elec_Equipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet Net Elec

Promotion DA continue - Dec. 2021 
Membres de l'équipe : Christine, Pierre-Emmanuel, Cédric
Mentor : Pierre

# Partie 1 : Chargement et Preprocessing <a class="anchor" id="Partie01"><a>

## 1.1 Chargements modules et connexion Google Drive

In [ ]:
# Montage du Drive à Colab pour hebergé le dataset

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Chargement des modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

## 1.2 Preprocessing DataSet Energie

### 1.2.1 Chargement

In [2]:
df_energie = pd.read_csv(filepath_or_buffer='eco2mix-regional-cons-def.csv', 
                 sep=';', 
                 parse_dates = {'Date_Heure': ['Date', 'Heure'] })

C:\Users\Christine\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (27,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#df_energie.head(5)

### 1.2.2 Filtrage

In [5]:
df_energie.columns

Index(['Date_Heure', 'Code INSEE région', 'Région', 'Nature', 'Date - Heure',
       'Consommation (MW)', 'Thermique (MW)', 'Nucléaire (MW)', 'Eolien (MW)',
       'Solaire (MW)', 'Hydraulique (MW)', 'Pompage (MW)', 'Bioénergies (MW)',
       'Ech. physiques (MW)', 'Stockage batterie', 'Déstockage batterie',
       'Eolien terrestre', 'Eolien offshore', 'TCO Thermique (%)',
       'TCH Thermique (%)', 'TCO Nucléaire (%)', 'TCH Nucléaire (%)',
       'TCO Eolien (%)', 'TCH Eolien (%)', 'TCO Solaire (%)',
       'TCH Solaire (%)', 'TCO Hydraulique (%)', 'TCH Hydraulique (%)',
       'TCO Bioénergies (%)', 'TCH Bioénergies (%)', 'Column 30'],
      dtype='object')

In [6]:
### Liste de varaibles à supprimer
col_to_drop = ['Column 30',
              'Code INSEE région',
              'Date_Heure'
              ]
### Suppression des varaibles inutiles
df_energie = df_energie.drop(col_to_drop, axis=1)

In [ ]:
# df_energie.info()

In [7]:
# Filtrage temporel

# Conversion du type pour le champs date
df_energie['Date - Heure'] = pd.to_datetime(df_energie['Date - Heure'], utc=True)

# Limites temporelles haute et basse
df_energie = df_energie[(df_energie['Date - Heure'] >= '01/01/2016') & (df_energie['Date - Heure'] <= '31/12/2021')]

### 1.2.3 Standardisation

In [8]:
# Les champs sont renommés : 
Dic_Noms = {
    'Région'             :'region_nom',
    'Nature'             :'nature',
    'Date - Heure'       :'date',
    'Consommation (MW)'  :'conso_inter',
    'Pompage (MW)'       :'conso_hydro',
    'Thermique (MW)'     :'prod_therm',
    'Nucléaire (MW)'     :'prod_nuc',
    'Solaire (MW)'       :'prod_sol',
    'Hydraulique (MW)'   :'prod_hydro',
    'Bioénergies (MW)'   :'prod_bio',
    'Eolien (MW)'        :'prod_eol',
    'Ech. physiques (MW)':'echanges',
    'TCO Thermique (%)'  :'tco_therm', 
    'TCO Nucléaire (%)'  :'tco_nuc',
    'TCO Eolien (%)'     :'tco_eol', 
    'TCO Solaire (%)'    :'tco_sol', 
    'TCO Hydraulique (%)':'tco_hydro',
    'TCO Bioénergies (%)':'tco_bio', 
    'TCH Nucléaire (%)'  :'tch_nuc', 
    'TCH Eolien (%)'     :'tch_eol',
    'TCH Thermique (%)'  :'tch_therm', 
    'TCH Solaire (%)'    :'tch_sol', 
    'TCH Hydraulique (%)':'tch_hydro', 
    'TCH Bioénergies (%)':'tch_bio', 
            }

df_energie = df_energie.rename(Dic_Noms, axis=1)

### 1.2.4 Complétude (Doublons & NANs)

In [11]:
# Identification des lignes dupliquées
df_energie[df_energie.duplicated()]

,region_nom,nature,date,conso_inter,prod_therm,prod_nuc,prod_eol,prod_sol,prod_hydro,conso_hydro,...,tco_nuc,tch_nuc,tco_eol,tch_eol,tco_sol,tch_sol,tco_hydro,tch_hydro,tco_bio,tch_bio
680309,Île-de-France,Données définitives,2016-03-27 01:00:00+00:00,7215.0,298.0,NaN,4.0,0.0,11.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
680311,Bourgogne-Franche-Comté,Données définitives,2016-03-27 01:00:00+00:00,2181.0,107.0,NaN,297.0,0.0,86.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
680315,Hauts-de-France,Données définitives,2016-03-27 01:00:00+00:00,5591.0,410.0,5301.0,551.0,0.0,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
680316,Occitanie,Données définitives,2016-03-27 01:00:00+00:00,3889.0,56.0,2337.0,298.0,0.0,836.0,-670.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
680319,Nouvelle-Aquitaine,Données définitives,2016-03-27 01:00:00+00:00,4592.0,156.0,3677.0,245.0,-1.0,465.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732673,Nouvelle-Aquitaine,Données consolidées,2021-03-28 01:00:00+00:00,5008.0,145.0,4101.0,131.0,0.0,202.0,NaN,...,0.0,0.0,4.03,11.44,2.22,32.74,NaN,NaN,NaN,NaN
1732675,Île-de-France,Données consolidées,2021-03-28 01:00:00+00:00,7017.0,271.0,NaN,41.0,0.0,11.0,NaN,...,0.0,0.0,0.16,55.00,2.18,47.81,NaN,NaN,NaN,NaN
1732677,Pays de la Loire,Données consolidées,2021-03-28 01:00:00+00:00,3232.0,156.0,NaN,53.0,0.0,4.0,NaN,...,0.0,0.0,0.12,36.36,1.55,64.94,NaN,NaN,NaN,NaN
1732678,Normandie,Données consolidées,2021-03-28 01:00:00+00:00,3187.0,254.0,8189.0,266.0,0.0,10.0,NaN,...,0.0,0.0,0.31,33.33,2.67,66.93,NaN,NaN,NaN,NaN


In [12]:
# Traitement des lignes dupliquées
df_energie = df_energie.drop_duplicates()

In [13]:
# Identification des NANs
df_energie.isna().sum(axis = 0)

region_nom                   0
nature                       0
date                         0
conso_inter                  0
prod_therm                   0
prod_nuc                508248
prod_eol                     0
prod_sol                     0
prod_hydro                   0
conso_hydro             525815
prod_bio                     0
echanges                     0
Stockage batterie      1017134
Déstockage batterie     999567
Eolien terrestre       1139638
Eolien offshore        1139638
tco_therm               841466
tch_therm               841464
tco_nuc                 911734
tch_nuc                 929299
tco_eol                 841466
tch_eol                 841464
tco_sol                 841466
tch_sol                 841464
tco_hydro              1051152
tch_hydro              1051152
tco_bio                1051152
tch_bio                1051152
dtype: int64

In [14]:
# Traitement des NANs:

# Cas 1 : Suppression des lignes
df_energie.dropna(subset = ['conso_inter'], axis=0, inplace = True)

# Cas 2 : Remplacement par zéro
list_fillzero=['prod_nuc', 'prod_therm', 'prod_eol', 'prod_sol', 'prod_hydro', 'prod_bio', 'conso_hydro']

for i in list_fillzero:
    df_energie[i] = df_energie[i].fillna(0)

# Cas 3 : Remplacement par zéro
list_fillzero=['tco_therm', 'tch_therm', 'tco_nuc', 'tch_nuc', 'tco_eol', 'tch_eol', 'tco_sol','tch_sol','tco_hydro','tch_hydro','tco_bio','tch_bio']

for i in list_fillzero:
    df_energie[i] = df_energie[i].fillna(0)
    

In [15]:
df_energie.isna().sum(axis = 0)

region_nom                   0
nature                       0
date                         0
conso_inter                  0
prod_therm                   0
prod_nuc                     0
prod_eol                     0
prod_sol                     0
prod_hydro                   0
conso_hydro                  0
prod_bio                     0
echanges                     0
Stockage batterie      1017134
Déstockage batterie     999567
Eolien terrestre       1139638
Eolien offshore        1139638
tco_therm                    0
tch_therm                    0
tco_nuc                      0
tch_nuc                      0
tco_eol                      0
tch_eol                      0
tco_sol                      0
tch_sol                      0
tco_hydro                    0
tch_hydro                    0
tco_bio                      0
tch_bio                      0
dtype: int64

### 1.2.5 Standardisation temporelle avec le DataSet Météo

Probleme : pour grouper les TCO et TCH il faut les convertir de % en MW

In [ ]:
# Groupby : Afin d'avoir la même granularité que la variable temps du dataset météo.
# Nous perdons de l'information, mais je ne sais pas comment ré-echantillonner le DF météo afin de dedoubler les lignes.

df_energie['date_index'] = df_energie['date']
df_energie = df_energie.set_index('date_index')

df_energie = df_energie.groupby( ['region_nom','nature', pd.Grouper(key = 'date' , freq = 'H')]).agg({
    'conso_inter' : 'sum', 
    'prod_therm'  : 'sum', 
    'prod_nuc'    : 'sum', 
    'prod_eol'    : 'sum', 
    'prod_sol'    : 'sum',
    'prod_hydro'  : 'sum', 
    'conso_hydro' : 'sum', 
    'prod_bio'    : 'sum', 
    'echanges'    : 'sum',
})


df_energie = df_energie.reset_index()

df_energie = df_energie.sort_values(by = ['date'], ascending = True)

## 1.3 Preprocessing DataSet Meteo

### 1.3.1 Chargement

In [57]:
df_meteo = pd.read_csv(filepath_or_buffer='donnees-synop-essentielles-omm.csv', 
                 sep=';')

### 1.3.2 Filtrage

In [58]:
# Suppression des colonnes inutiles
# Dans un premier temps on ne conserve que la temperature pour simplifier le dataset

List_cols = {'ID OMM station', 
             'Pression au niveau mer',
             'Variation de pression en 3 heures', 
             'Type de tendance barométrique',
             'Direction du vent moyen 10 mn', 
             'Point de rosée', 
             'Humidité', 
             'Visibilité horizontale',
             'Temps présent', 
             'Temps passé 1', 
             'Temps passé 2', 
             "Nébulosité  des nuages de l' étage inférieur",
             "Hauteur de la base des nuages de l'étage inférieur",
             "Type des nuages de l'étage inférieur",
             "Type des nuages de l'étage moyen",
             "Type des nuages de l'étage supérieur", 
             'Pression station',
             'Niveau barométrique', 
             'Géopotentiel',
             'Variation de pression en 24 heures',
             'Température minimale sur 12 heures',
             'Température minimale sur 24 heures',
             'Température maximale sur 12 heures',
             'Température maximale sur 24 heures',
             'Température minimale du sol sur 12 heures',
             'Méthode de mesure Température du thermomètre mouillé',
             'Température du thermomètre mouillé',
             'Rafale sur les 10 dernières minutes', 
             'Rafales sur une période',
             'Periode de mesure de la rafale', 
             'Etat du sol',
             'Hauteur totale de la couche de neige, glace, autre au sol',
             'Hauteur de la neige fraîche', 
             'Periode de mesure de la neige fraiche',
             'Précipitations dans la dernière heure',
             'Précipitations dans les 3 dernières heures',
             'Précipitations dans les 6 dernières heures',
             'Précipitations dans les 12 dernières heures',
             'Précipitations dans les 24 dernières heures', 
             'Phénomène spécial 1',
             'Phénomène spécial 2', 
             'Phénomène spécial 3', 
             'Phénomène spécial 4',
             'Nébulosité couche nuageuse 1', 
             'Type nuage 1', 
             'Hauteur de base 1',
             'Nébulosité couche nuageuse 2', 
             'Type nuage 2', 
             'Hauteur de base 2',
             'Nébulosité couche nuageuse 3', 
             'Type nuage 3', 
             'Hauteur de base 3',
             'Nébulosité couche nuageuse 4', 
             'Type nuage 4', 
             'Hauteur de base 4',
             'Coordonnees', 
             'Nom', 
             'Type de tendance barométrique.1',
             'Temps passé 1.1', 
             'Temps présent.1', 
             'Température (°C)',
             'Température minimale sur 12 heures (°C)',
             'Température minimale sur 24 heures (°C)',
             'Température maximale sur 12 heures (°C)',
             'Température maximale sur 24 heures (°C)',
             'Température minimale du sol sur 12 heures (en °C)', 'Latitude',
             'Longitude', 
             'Altitude', 
             'communes (name)', 
             'communes (code)',
             'EPCI (name)', 
             'EPCI (code)', 
             'department (name)', 
             'department (code)',
             'region (code)',
             'mois_de_l_annee'
            }

df_meteo = df_meteo.drop(List_cols, axis=1)

In [ ]:
# Identification des région à supprimer
# df_meteo['region (name)'].unique()

In [59]:
# Suppression des région qui sont en dehors du dataset energie
region_to_drop = ['Martinique',
                  'Guyane',
                  'La Réunion',
                  'Guadeloupe',
                  'Mayotte',
                  'Terres australes et antarctiques françaises',
                  'Saint-Pierre-et-Miquelon',
                  'Saint-Barthélemy',
                   'Corse'
                 ]

for j in region_to_drop:
    df_meteo = df_meteo[df_meteo['region (name)'] != j]

In [ ]:
# Contrôle
# df_meteo['region (name)'].unique()

### 1.3.3 Standardisation

In [60]:
# Les variables sont renommées pour plus d'homogénéité avec le 1er DataFrame

Dic_Noms_meteo = {'Date'             : 'date',
                 'Température'       : 'temperature',
                 'region (name)'     : 'region_nom', 
                 'Vitesse du vent moyen 10 mn': 'vitesse_vent_10min',
                 'Nebulosité totale' : 'nebulosite_totale'
                }
    
df_meteo = df_meteo.rename(Dic_Noms_meteo, axis=1)
df_meteo.columns

Index(['date', 'vitesse_vent_10min', 'temperature', 'nebulosite_totale',
       'region_nom'],
      dtype='object')

In [61]:
# Temperature : conversion degrès Kevin => degrès Celsius
df_meteo['temperature'] = df_meteo['temperature'] - 273.15

In [46]:
# date : Conversion du type de variable temps
df_meteo['date'] = pd.to_datetime(df_meteo['date'], utc=True)

In [63]:
# Standardisation des noms des régions
dico_df_meteo = {'Occitanie'           :'Occitanie',
                 'Normandie'           :'Normandie', 
                 'Bretagne'            :'Bretagne',
                 "Provence-Alpes-Côte d'Azur":"Provence-Alpes-Côte d'Azur", 
                 'Hauts-de-France'     :'Hauts-de-France', 
                 'Île-de-France'       :'Île-de-France',
                 'Grand Est'           :'Grand Est', 
                 'Nouvelle-Aquitaine'  :'Nouvelle-Aquitaine',
                 'Bourgogne-Franche-Comté':'Bourgogne-Franche-Comté', 
                 'Pays de la Loire'    :'Centre-Val de Loire',
                 'Auvergne-Rhône-Alpes':'Auvergne-Rhône-Alpes', 
                 'Centre-Val de Loire' :'Centre-Val de Loire'}

df_meteo = df_meteo.rename(dico_df_meteo, axis = 1)

### 1.3.4 Complétude

**Doublons**

In [64]:
# Recherche de lignes dupliquées
df_meteo.duplicated().sum()

764

In [65]:
# Traitement des lignes en doublons
df_meteo = df_meteo.drop_duplicates()

**Region NAN**

In [66]:
# Correction de la région 'nan'
df_meteo[df_meteo['region_nom'].isna()==True].head(5)

,date,vitesse_vent_10min,temperature,nebulosite_totale,region_nom
8,2013-04-03T23:00:00+02:00,1.0,23.5,NaN,NaN
15,2013-04-05T23:00:00+02:00,0.0,23.5,NaN,NaN
20,2013-04-06T05:00:00+02:00,6.2,26.8,NaN,NaN
25,2013-04-06T08:00:00+02:00,0.0,-16.3,90.0,NaN
29,2013-04-06T11:00:00+02:00,11.8,5.6,NaN,NaN


In [91]:
# suppression de la région NAN
df_meteo.dropna(subset = ['region_nom'], axis=0, inplace = True)

**Autres NANs**

In [95]:
df_meteo.sort_index().sort_values('region_nom')

,vitesse_vent_10min,temperature,nebulosite_totale,region_nom
date,,,,
2018-12-16 06:00:00+00:00,0.0,3.0,77.05043,Auvergne-Rhône-Alpes
2021-09-27 03:00:00+00:00,2.4,13.9,90.00000,Auvergne-Rhône-Alpes
2020-01-31 03:00:00+00:00,2.2,8.8,100.00000,Auvergne-Rhône-Alpes
2016-01-01 09:00:00+00:00,3.7,9.3,50.00000,Auvergne-Rhône-Alpes
2020-01-31 03:00:00+00:00,3.9,12.1,100.00000,Auvergne-Rhône-Alpes
...,...,...,...,...
2013-03-31 15:00:00+00:00,3.6,5.0,75.00000,Île-de-France
2010-08-29 06:00:00+00:00,3.1,12.1,60.00000,Île-de-France
2019-11-17 03:00:00+00:00,1.5,5.1,100.00000,Île-de-France


In [75]:
# date : Conversion du type de variable temps
df_meteo['date'] = pd.to_datetime(df_meteo['date'], utc=True)

In [77]:
df_meteo = df_meteo.set_index('date')

In [78]:
col_meteo = df_meteo.columns

In [87]:
df_meteo[df_meteo.columns[:3]] = imputer.fit_transform(df_meteo.iloc[:, :3])

In [89]:
df_meteo.isna().sum()

vitesse_vent_10min         0
temperature                0
nebulosite_totale          0
region_nom            223834
dtype: int64

In [ ]:
# Traitement des NANs des variables quantitatives

df_meteo['temperature']        = df_meteo['temperature'].fillna(method='pad') #Remplacement par la valeur précédente
df_meteo['vitesse_vent_10min'] = df_meteo['vitesse_vent_10min'].fillna(method='pad') #Remplacement par la valeur précédente
df_meteo['nebulosite_totale']  = df_meteo['nebulosite_totale'].fillna(method='pad') #Remplacement par la valeur précédente

#df_meteo['temperature']        = df_meteo['temperature'].fillna(df_meteo['temperature'].mean())
#df_meteo['vitesse_vent_10min'] = df_meteo['vitesse_vent_10min'].fillna(df_meteo['vitesse_vent_10min'].mean())
#df_meteo['nebulosite_totale']  = df_meteo['nebulosite_totale'].fillna(df_meteo['nebulosite_totale'].mean())

In [97]:
df_meteo.isna().sum(axis = 0)

vitesse_vent_10min    0
temperature           0
nebulosite_totale     0
region_nom            0
dtype: int64

In [104]:
df_meteo = df_meteo.reset_index().groupby(['region_nom', 'date'])[['temperature', 'nebulosite_totale', 'region_nom']].mean()

In [105]:
df_meteo

temperature  nebulosite_totale
region_nom           date                                                     
Auvergne-Rhône-Alpes 2010-01-01 00:00:00+00:00        6.300          86.025215
                     2010-01-01 03:00:00+00:00        5.725          86.025215
                     2010-01-01 06:00:00+00:00        5.700          97.500000
                     2010-01-01 09:00:00+00:00        5.925         100.000000
                     2010-01-01 12:00:00+00:00        5.875         100.000000
...                                                     ...                ...
Île-de-France        2022-05-23 06:00:00+00:00       16.300         100.000000
                     2022-05-23 09:00:00+00:00       18.900          90.000000
                     2022-05-23 12:00:00+00:00       16.700         100.000000
                     2022-05-23 15:00:00+00:00       17.300          90.000000
                     2022-05-23 18:00:00+00:00       15.800         100.000000

[433898 rows x 2 columns]

In [108]:
df_energie = df_energie.groupby(['region_nom', 'date']).mean()

## 1.4 Fusion des deux DataSet

### 1.4.1 Merge

In [126]:
# jointure des deux dataset

display(df_energie.head())
display(df_meteo.head())

conso_inter  prod_therm  \
region_nom           date                                                 
Auvergne-Rhône-Alpes 2016-01-01 00:00:00+00:00       7789.0       193.0   
                     2016-01-01 00:30:00+00:00       7749.0       193.0   
                     2016-01-01 01:00:00+00:00       7724.0       193.0   
                     2016-01-01 01:30:00+00:00       7775.0       193.0   
                     2016-01-01 02:00:00+00:00       7580.0       194.0   

                                                prod_nuc  prod_eol  prod_sol  \
region_nom           date                                                      
Auvergne-Rhône-Alpes 2016-01-01 00:00:00+00:00   11472.0       5.0       0.0   
                     2016-01-01 00:30:00+00:00   11541.0       6.0       0.0   
                     2016-01-01 01:00:00+00:00   11479.0       7.0       0.0   
                     2016-01-01 01:30:00+00:00   11547.0       7.0       0.0   
                     2016-01-01 02:00:00+00:00   11428.0       7.0       0.0   

                                                prod_hydro  conso_hydro  \
region_nom           date                                                 
Auvergne-Rhône-Alpes 2016-01-01 00:00:00+00:00      1303.0         -3.0   
                     2016-01-01 00:30:00+00:00      1747.0         -4.0   
                     2016-01-01 01:00:00+00:00      1450.0         -4.0   
                     2016-01-01 01:30:00+00:00      1022.0        -12.0   
                     2016-01-01 02:00:00+00:00       802.0       -891.0   

                                                prod_bio  echanges  \
region_nom           date                                            
Auvergne-Rhône-Alpes 2016-01-01 00:00:00+00:00     130.0   -5311.0   
                     2016-01-01 00:30:00+00:00     120.0   -5854.0   
                     2016-01-01 01:00:00+00:00     131.0   -5533.0   
                     2016-01-01 01:30:00+00:00     130.0   -5112.0   
                     2016-01-01 02:00:00+00:00     130.0   -4089.0   

                                                Stockage batterie  ...  \
region_nom           date                                          ...   
Auvergne-Rhône-Alpes 2016-01-01 00:00:00+00:00                NaN  ...   
                     2016-01-01 00:30:00+00:00                NaN  ...   
                     2016-01-01 01:00:00+00:00                NaN  ...   
                     2016-01-01 01:30:00+00:00                NaN  ...   
                     2016-01-01 02:00:00+00:00                NaN  ...   

                                                tco_therm  tch_therm  tco_nuc  \
region_nom           date                                                       
Auvergne-Rhône-Alpes 2016-01-01 00:00:00+00:00        0.0        0.0      0.0   
                     2016-01-01 00:30:00+00:00        0.0        0.0      0.0   
                     2016-01-01 01:00:00+00:00        0.0        0.0      0.0   
                     2016-01-01 01:30:00+00:00        0.0        0.0      0.0   
                     2016-01-01 02:00:00+00:00        0.0        0.0      0.0   

                                                tch_nuc  tco_eol  tch_eol  \
region_nom           date                                                   
Auvergne-Rhône-Alpes 2016-01-01 00:00:00+00:00      0.0      0.0      0.0   
                     2016-01-01 00:30:00+00:00      0.0      0.0      0.0   
                     2016-01-01 01:00:00+00:00      0.0      0.0      0.0   
                     2016-01-01 01:30:00+00:00      0.0      0.0      0.0   
                     2016-01-01 02:00:00+00:00      0.0      0.0      0.0   

                                                tco_sol  tch_sol  tch_hydro  \
region_nom           date                                                     
Auvergne-Rhône-Alpes 2016-01-01 00:00:00+00:00      0.0      0.0        0.0   
                     2016-01-01 00:30:00+00:00      0.0      0.0        0.0   
 

temperature  nebulosite_totale
region_nom           date                                                     
Auvergne-Rhône-Alpes 2010-01-01 00:00:00+00:00        6.300          86.025215
                     2010-01-01 03:00:00+00:00        5.725          86.025215
                     2010-01-01 06:00:00+00:00        5.700          97.500000
                     2010-01-01 09:00:00+00:00        5.925         100.000000
                     2010-01-01 12:00:00+00:00        5.875         100.000000

In [127]:
df = df_energie.join(df_meteo)

In [130]:
df.head(20)

conso_inter  prod_therm  \
region_nom           date                                                 
Auvergne-Rhône-Alpes 2016-01-01 00:00:00+00:00       7789.0       193.0   
                     2016-01-01 00:30:00+00:00       7749.0       193.0   
                     2016-01-01 01:00:00+00:00       7724.0       193.0   
                     2016-01-01 01:30:00+00:00       7775.0       193.0   
                     2016-01-01 02:00:00+00:00       7580.0       194.0   
                     2016-01-01 02:30:00+00:00       7475.0       193.0   
                     2016-01-01 03:00:00+00:00       7312.0       193.0   
                     2016-01-01 03:30:00+00:00       7196.0       193.0   
                     2016-01-01 04:00:00+00:00       7109.0       193.0   
                     2016-01-01 04:30:00+00:00       7123.0       193.0   
                     2016-01-01 05:00:00+00:00       7084.0       189.0   
                     2016-01-01 05:30:00+00:00       7013.0       189.0   
                     2016-01-01 06:00:00+00:00       6964.0       190.0   
                     2016-01-01 06:30:00+00:00       6904.0       193.0   
                     2016-01-01 07:00:00+00:00       6952.0       193.0   
                     2016-01-01 07:30:00+00:00       6839.0       193.0   
                     2016-01-01 08:00:00+00:00       6902.0       194.0   
                     2016-01-01 08:30:00+00:00       7121.0       194.0   
                     2016-01-01 09:00:00+00:00       7229.0       194.0   
                     2016-01-01 09:30:00+00:00       7300.0       194.0   

                                                prod_nuc  prod_eol  prod_sol  \
region_nom           date                                                      
Auvergne-Rhône-Alpes 2016-01-01 00:00:00+00:00   11472.0       5.0       0.0   
                     2016-01-01 00:30:00+00:00   11541.0       6.0       0.0   
                     2016-01-01 01:00:00+00:00   11479.0       7.0       0.0   
                     2016-01-01 01:30:00+00:00   11547.0       7.0       0.0   
                     2016-01-01 02:00:00+00:00   11428.0       7.0       0.0   
                     2016-01-01 02:30:00+00:00   11014.0      10.0       0.0   
                     2016-01-01 03:00:00+00:00   10985.0      19.0       0.0   
                     2016-01-01 03:30:00+00:00   11024.0      25.0       0.0   
                     2016-01-01 04:00:00+00:00   10947.0      40.0       0.0   
                     2016-01-01 04:30:00+00:00   10985.0      44.0       0.0   
                     2016-01-01 05:00:00+00:00   11009.0      44.0       0.0   
                     2016-01-01 05:30:00+00:00   11049.0      48.0       0.0   
                     2016-01-01 06:00:00+00:00   11031.0      49.0       0.0   
                     2016-01-01 06:30:00+00:00   10973.0      54.0       0.0   
                     2016-01-01 07:00:00+00:00   10916.0      54.0       0.0   
                     2016-01-01 07:30:00+00:00   10597.0      66.0       0.0   
                     2016-01-01 08:00:00+00:00   10630.0      72.0       9.0   
                     2016-01-01 08:30:00+00:00   10948.0      71.0      33.0   
                     2016-01-01 09:00:00+00:00   10913.0      61.0      70.0   
                     2016-01-01 09:30:00+00:00   10954.0      52.0      96.0   

                                                prod_hydro  conso_hydro  \
region_nom           date                                                 
Auvergne-Rhône-Alpes 2016-01-01 00:00:00+00:00      1303.0         -3.0   
                     2016-01-01 00:30:00+00:00      1747.0         -4.0   
                     2016-01-01 01:00:00+00:00      1450.0         -4.0   
                     2016-01-01 01:30:00+00:00      1022.0        -12.0   
                     2016-01-01 02:00:00+00:00       802.0       -891.0   
                     2016-01-01 02:30:00+00:00       810.0      -1775.0   
                     2016-01-01 03:0

### 1.4.2 Complétude

In [153]:
df[df.columns[-2:]] = df.iloc[:, -2:].interpolate()

In [156]:
df.isna().sum()

conso_inter                  0
prod_therm                   0
prod_nuc                     0
prod_eol                     0
prod_sol                     0
prod_hydro                   0
conso_hydro                  0
prod_bio                     0
echanges                     0
Stockage batterie      1017076
Déstockage batterie     999510
Eolien terrestre       1139573
Eolien offshore        1139573
tco_therm                    0
tch_therm                    0
tco_nuc                      0
tch_nuc                      0
tco_eol                      0
tch_eol                      0
tco_sol                      0
tch_sol                      0
tch_hydro                    0
tch_bio                      0
temperature                  0
nebulosite_totale            0
dtype: int64

# Partie 2 : Exploration et Story-telling <a class="anchor" id="Partie02"><a>

## 2.1 Exploration sur la consommation d'energie

## 2.2 Exploration sur les sources de productions

## 2.3 Relations Consommation & Production

## 2.4 Relations consommation & Météo

# Partie 3 : Problématique et analyse <a class="anchor" id="Partie03"><a>

## 3.1 Définition de la problématique

## 3.2 Construction d'indicateurs

## 3.3 Données pour Dashboard PowerBI

## 3.4 Prédictions

# Partie 4 : Conclusions  <a class="anchor" id="Partie04"><a>

## 4.1 Apports du projet en réponse à la problématique

## 4.2 Limites et axes d'amélioration